In [ ]:
!unzip -q '/content/drive/MyDrive/oto_recommend/data/otto-validation.zip'

In [ ]:
!kaggle datasets download -d alexlods/ottoprecalculatedfeatureparquet
!unzip -q /content/ottoprecalculatedfeatureparquet.zip

 99% 469M/475M [00:04<00:00, 123MB/s]
100% 475M/475M [00:04<00:00, 118MB/s]


In [ ]:
import pandas as pd
import glob
import datetime,time
from tqdm.notebook import tqdm
from  datetime  import datetime
!pip install -q pandarallel
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import gc
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold,GroupKFold
import lightgbm as lgb
import joblib

In [ ]:
embedding_weight = np.load('/content/word2vec.model.wv.vectors.npy')
aid_embedding = pd.DataFrame(embedding_weight, columns = ['featuress8_Embedding_' + str(x) for x in range(32)]).reset_index().rename(columns = {'index':'aid'})

In [ ]:
df_train_dict = {}
objects = ['clicks','carts','orders']
for t in objects:
  df_train_dict[t] = pd.read_parquet(f'/content/drive/MyDrive/oto_recommend/process_data/train_data_{t}_v2.parquet')

In [ ]:
last_click_embedding =  pd.DataFrame(embedding_weight, columns = ['featuress9_last_click_Embedding_' + str(x) for x in range(32)]).reset_index().rename(columns = {'index':'aid'})
last_carts_embedding =  pd.DataFrame(embedding_weight, columns = ['featuress9_last_carts_Embedding_' + str(x) for x in range(32)]).reset_index().rename(columns = {'index':'aid'})
last_orders_embedding =  pd.DataFrame(embedding_weight, columns = ['featuress9_last_orders_Embedding_' + str(x) for x in range(32)]).reset_index().rename(columns = {'index':'aid'})
for t in objects:
  df_train_dict[t] = df_train_dict[t].merge(last_click_embedding,how='left',left_on = ['last_clicks_3_aid'] , right_on=['aid'])
  # df_train_dict[t] = df_train_dict[t].merge(last_carts_embedding,how='left',left_on = ['last_carts_1_aid'] , right_on=['aid'])
  # df_train_dict[t] = df_train_dict[t].merge(last_orders_embedding,how='left',left_on = ['last_orders_1_aid'] , right_on=['aid'])

In [ ]:
features =  [i for i in df_train_dict['clicks'].columns if 'features' in i and 'featuress9' not in i and 'featuress8' not in i] + ['all_clicks_count','all_carts_count','all_orders_count'] 

In [ ]:
# for t in objects:
#   tmp = df_train_dict[t]
#   for p in features:
#     tmp[p] = (tmp[p] -  tmp[p].min()) / (tmp[p].max() - tmp[p].min())
#   df_train_dict[t] = tmp

In [ ]:
def train_model(t):
  result = []
  df_train = df_train_dict[t]
  features =  [i for i in df_train.columns if 'features' in i] + ['all_clicks_count','all_carts_count','all_orders_count'] 
  kf = GroupKFold(n_splits=5)
  models = []
  for fold , (train_index,test_index) in enumerate(kf.split(df_train[features],df_train['label'],groups=df_train['session'])):
      # xgb = XGBClassifier(n_estimators = 10000,tree_method='gpu_hist',objective='rank:pairwise',learning_rate=0.6)
      df_train_x , df_train_y = df_train.loc[train_index,features],df_train.loc[train_index,'label']
      df_test_x , df_test_y = df_train.loc[test_index,features],df_train.loc[test_index,'label'].values
      dtrain = xgb.DMatrix(df_train_x,df_train_y) 
      dvalid = xgb.DMatrix(df_test_x, df_test_y) 

      xgb_parms = {'objective':'rank:pairwise', 'tree_method':'gpu_hist','eval_metric':'auc'}
      model = xgb.train(xgb_parms, dtrain=dtrain,evals=[(dtrain,'train'),(dvalid,'valid')],num_boost_round=10000,verbose_eval=100,early_stopping_rounds=30)
      # xgb.fit(df_train_x,df_train_y,eval_set = [(df_train_x,df_train_y),(df_test_x,df_test_y)],
      #       eval_metric = ["auc"],
      #       early_stopping_rounds = 10,
      #       verbose = 100)
      joblib.dump(model,f'/content/drive/MyDrive/oto_recommend/model/xgb_model_v2_{t}_{fold}_rank_pairwise.pkl')
      for _ in range(5):
        gc.collect()

In [ ]:
import xgboost as xgb

In [ ]:
models_carts = train_model('carts')

[0]	train-auc:0.763632	valid-auc:0.762254
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[100]	train-auc:0.839725	valid-auc:0.826272
[200]	train-auc:0.858034	valid-auc:0.833029
[300]	train-auc:0.871268	valid-auc:0.834368
Stopping. Best iteration:
[353]	train-auc:0.877309	valid-auc:0.834609

[0]	train-auc:0.764196	valid-auc:0.762092
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[100]	train-auc:0.841023	valid-auc:0.826594
[200]	train-auc:0.858886	valid-auc:0.832613
[300]	train-auc:0.87224	valid-auc:0.834065
Stopping. Best iteration:
[319]	train-auc:0.874454	valid-auc:0.834213

[0]	train-auc:0.764936	valid-auc:0.764068
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[100]	train-auc:0.839878	valid-auc:0.825

In [ ]:
models_orders = train_model('orders')

[0]	train-auc:0.782288	valid-auc:0.782337
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[100]	train-auc:0.86972	valid-auc:0.858834
[200]	train-auc:0.886997	valid-auc:0.866476
Stopping. Best iteration:
[240]	train-auc:0.891581	valid-auc:0.867045

[0]	train-auc:0.781197	valid-auc:0.781028
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[100]	train-auc:0.86929	valid-auc:0.858432
[200]	train-auc:0.88698	valid-auc:0.866926
Stopping. Best iteration:
[264]	train-auc:0.894247	valid-auc:0.867813

[0]	train-auc:0.782576	valid-auc:0.779868
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[100]	train-auc:0.868844	valid-auc:0.856892
[200]	train-auc:0.886152	valid-auc:0.864305
[300]	train-auc:0.897682	valid-auc:0.86621

In [ ]:
models_clicks = train_model('clicks')

[0]	train-auc:0.767283	valid-auc:0.766948
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[100]	train-auc:0.817447	valid-auc:0.813836
[200]	train-auc:0.828463	valid-auc:0.821714
[300]	train-auc:0.835229	valid-auc:0.824873
[400]	train-auc:0.840783	valid-auc:0.827037
[500]	train-auc:0.845549	valid-auc:0.828354
[600]	train-auc:0.850055	valid-auc:0.829544
[700]	train-auc:0.854017	valid-auc:0.830216
[800]	train-auc:0.857721	valid-auc:0.830746
[900]	train-auc:0.861313	valid-auc:0.831234
[1000]	train-auc:0.864616	valid-auc:0.831602
[1100]	train-auc:0.867743	valid-auc:0.83195
[1200]	train-auc:0.870756	valid-auc:0.832091
Stopping. Best iteration:
[1189]	train-auc:0.870442	valid-auc:0.83213

[0]	train-auc:0.767474	valid-auc:0.766737
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[100]	train-auc:0.816555	vali

In [ ]:
models_clicks = train_model('clicks')

[0]	train-auc:0.765727	valid-auc:0.765258
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[100]	train-auc:0.798206	valid-auc:0.794312
[200]	train-auc:0.807126	valid-auc:0.800027
[300]	train-auc:0.813427	valid-auc:0.803024
[400]	train-auc:0.818585	valid-auc:0.804844
[500]	train-auc:0.823085	valid-auc:0.806117
[600]	train-auc:0.827009	valid-auc:0.806984
[700]	train-auc:0.830693	valid-auc:0.807728
[800]	train-auc:0.834152	valid-auc:0.808258
[900]	train-auc:0.837324	valid-auc:0.808651
[1000]	train-auc:0.840293	valid-auc:0.808919
[1100]	train-auc:0.843017	valid-auc:0.809104
[1200]	train-auc:0.845696	valid-auc:0.809279
[1300]	train-auc:0.848129	valid-auc:0.809366
[1400]	train-auc:0.850656	valid-auc:0.809493
Stopping. Best iteration:
[1432]	train-auc:0.851393	valid-auc:0.809519

[0]	train-auc:0.765996	valid-auc:0.765279
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopp

In [ ]:
models_carts = train_model('carts')

[0]	train-auc:0.761486	valid-auc:0.760168
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[100]	train-auc:0.817292	valid-auc:0.804208
[200]	train-auc:0.832539	valid-auc:0.80752
[300]	train-auc:0.844195	valid-auc:0.808752
[400]	train-auc:0.854666	valid-auc:0.809341
Stopping. Best iteration:
[455]	train-auc:0.860011	valid-auc:0.809518

[0]	train-auc:0.762242	valid-auc:0.76011
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[100]	train-auc:0.818404	valid-auc:0.804029
[200]	train-auc:0.833742	valid-auc:0.807626
[300]	train-auc:0.84547	valid-auc:0.808743
[400]	train-auc:0.855932	valid-auc:0.809144
Stopping. Best iteration:
[434]	train-auc:0.859357	valid-auc:0.809283

[0]	train-auc:0.762917	valid-auc:0.761599
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train 

In [ ]:
def valid(): 
  result = {}
  objects = ['clicks','carts','orders']
  for t in objects:
    df_train = df_train_dict[t]
    features =  [i for i in df_train.columns if 'features' in i] + ['all_clicks_count','all_carts_count','all_orders_count']
    df_train['pred'] = -1 
    kf = GroupKFold(n_splits=5)
    # kf = StratifiedKFold(n_splits = 5)
    for fold , (train_index,test_index) in enumerate(kf.split(df_train[features],df_train['label'],groups=df_train['session'])):
        df_test_x , df_test_y = df_train.loc[test_index,features],df_train.loc[test_index,'label'].values
        model = joblib.load(f'/content/drive/MyDrive/oto_recommend/model/xgb_model_v2_{t}_{fold}_rank_pairwise.pkl')
        # pred = model.predict_proba(df_test_x)[:,1]
        dvalid = xgb.DMatrix(df_test_x , df_test_y) 
        pred = model.predict(dvalid)
        df_train.loc[test_index,'pred'] = pred 
    result[t] = df_train
  return result
df_valid = valid()

In [ ]:
tmp1 = df_valid['clicks']
tmp1['rank'] = df_valid['clicks'].groupby('session')['pred'].rank(ascending=False,method='first')
tmp1 = tmp1[tmp1['rank'] <= 20]
df_valid_clicks = tmp1.groupby('session')['recall_aids'].progress_apply(lambda x :list(set(x.values)))

tmp1 = df_valid['carts'] 
tmp1['rank'] = df_valid['carts'].groupby('session')['pred'].rank(ascending=False,method='first')
tmp1 = tmp1[tmp1['rank'] <= 20]
df_valid_carts = tmp1.groupby('session')['recall_aids'].progress_apply(lambda x :list(set(x.values)))

tmp1 = df_valid['orders'] 
tmp1['rank'] = df_valid['orders'].groupby('session')['pred'].rank(ascending=False,method='first')
tmp1 = tmp1[tmp1['rank'] <= 20]
df_valid_orders = tmp1.groupby('session')['recall_aids'].progress_apply(lambda x :list(set(x.values)))

100%|██████████| 149728/149728 [00:02<00:00, 70196.16it/s]


In [ ]:
result = {}
score = 0
weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}
object_list = ['clicks','carts','orders']
for t in object_list:
  if t == 'clicks':
    df_train = df_valid_clicks
  elif t == 'carts': 
    df_train = df_valid_carts 
  else:
    df_train = df_valid_orders
  test_pred_df = df_train.reset_index()
  test_pred_df.columns = ['session','pred_aid']
  test_label = pd.read_parquet('/content/test_labels.parquet')
  test_label_tmp = test_label[test_label['type'] == t]
  test_tmp = test_pred_df.merge(test_label_tmp[['session','ground_truth']],on='session')
  # test_tmp['recall_aid'] = test_tmp['aid'].progress_apply(lambda x : get_recall_aid(x,object=t))
  test_tmp['hits_aid'] = test_tmp.progress_apply(lambda df: list(set(df.ground_truth).intersection(df.pred_aid)), axis=1)
  test_tmp['hits'] = test_tmp['hits_aid'].apply(lambda x : len(x))
  test_tmp['gt_count'] = test_tmp.ground_truth.str.len().clip(0,20)
  recall = test_tmp['hits'].sum() / test_tmp['gt_count'].sum()
  score += weights[t]*recall
  print(f'{t} recall =',recall)
  result[t] = test_tmp
print('=============')
print('Overall Recall =',score)
print('=============')

100%|██████████| 1526566/1526566 [00:34<00:00, 44716.28it/s]


clicks recall = 0.6585218064597272


100%|██████████| 290322/290322 [00:06<00:00, 45652.63it/s]


carts recall = 0.4704708794158113


100%|██████████| 149728/149728 [00:03<00:00, 45212.63it/s]


orders recall = 0.6765662265792269
Overall Recall = 0.6129331804182523
